In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    Src['Datetime']                     = Query['datetime']
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])

    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']


    BINS_CHG = [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000, 5200, 5500]
    BINS_HL  = [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000]

    for x in BINS_CHG:  Calc[f'Chg Abs < {x}'] = (Calc['Chg Abs'] < x)
    for x in BINS_HL:   Calc[f'Hilo < {x}']    = (Calc['Hilo']    < x)


    FUTS = [1, 2]
    for x in FUTS:  Calc[f'Chg +{x}']       = Calc['Change'].shift(-x)
    for x in FUTS:  Calc[f'Chg Abs +{x}']   = Calc['Chg Abs'].shift(-x)
    for x in FUTS:  Calc[f'Chg Pos +{x}']   = Calc['Chg Pos'].shift(-x)
    for x in FUTS:  Calc[f'Chg Neg +{x}']   = Calc['Chg Neg'].shift(-x)

    for x in FUTS:  Calc[f'Hilo +{x}']      = Calc['Hilo'].shift(-x)
    for x in FUTS:  Calc[f'HL Pos +{x}']    = Calc['HL Pos'].shift(-x)
    for x in FUTS:  Calc[f'HL Neg +{x}']    = Calc['HL Neg'].shift(-x)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [ ]:
def MARKOV_CHAIN(Calc, when, _next):
    pipe = []
    for x in when: 
        Df = Calc[Calc[x]==1]

        for y in _next: 
            pipe.append({ 'Previous':x, 'Next':y, 
                'Count':          Df[y].count(), 
                'Avg':   np.round(Df[y].mean(),        0).astype(int),
                'Max':   np.round(Df[y].max(),         0).astype(int),
                'Q3':    np.round(Df[y].quantile(.75), 0).astype(int),
                'Med':   np.round(Df[y].median(),      0).astype(int),
                'Q1':    np.round(Df[y].quantile(.25), 0).astype(int),
                'Min':   np.round(Df[y].min(),         0).astype(int),
            })
        pass
    pass
    return pd.DataFrame(pipe)

In [24]:
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Chg Abs < {100}':f'Chg Abs < {5500}'].columns, _next=['Chg Abs +1'])

,Previous,Next,Count,Avg,Max,Q3,Med,Q1,Min
0,Chg Abs < 100,Chg Abs +1,78,1016,3985,1509,878,322,60
1,Chg Abs < 200,Chg Abs +1,148,1010,3985,1431,890,419,5
2,Chg Abs < 500,Chg Abs +1,367,1097,4295,1568,935,458,5
3,Chg Abs < 800,Chg Abs +1,587,1042,4295,1475,885,438,0
4,Chg Abs < 1000,Chg Abs +1,720,1040,4295,1475,865,425,0
5,Chg Abs < 1200,Chg Abs +1,833,1036,4295,1450,865,425,0
6,Chg Abs < 1500,Chg Abs +1,953,1024,4295,1440,855,415,0
7,Chg Abs < 1800,Chg Abs +1,1039,1016,4295,1430,850,412,0
8,Chg Abs < 2000,Chg Abs +1,1095,1017,5150,1430,845,415,0
9,Chg Abs < 2200,Chg Abs +1,1125,1014,5150,1430,840,410,0


In [23]:
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Chg Abs < {100}':f'Chg Abs < {5500}'].columns, _next=['Hilo +1'])

,Previous,Next,Count,Avg,Max,Q3,Med,Q1,Min
0,Chg Abs < 100,Hilo +1,78,1970,5205,2239,1745,1461,895
1,Chg Abs < 200,Hilo +1,148,1947,5205,2288,1762,1446,835
2,Chg Abs < 500,Hilo +1,367,1987,5205,2345,1800,1470,690
3,Chg Abs < 800,Hilo +1,587,1951,5205,2325,1790,1432,660
4,Chg Abs < 1000,Hilo +1,720,1952,5300,2318,1798,1435,610
5,Chg Abs < 1200,Hilo +1,833,1946,5300,2310,1790,1425,535
6,Chg Abs < 1500,Hilo +1,953,1934,5300,2310,1770,1425,535
7,Chg Abs < 1800,Hilo +1,1039,1930,5300,2310,1770,1422,535
8,Chg Abs < 2000,Hilo +1,1095,1937,5720,2325,1770,1410,535
9,Chg Abs < 2200,Hilo +1,1125,1937,5720,2330,1770,1410,535


In [22]:
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Hilo < {1000}':f'Hilo < {7500}'].columns, _next=['Chg Abs +1'])

,Previous,Next,Count,Avg,Max,Q3,Med,Q1,Min
0,Hilo < 1000,Chg Abs +1,60,984,3150,1405,920,354,15
1,Hilo < 1500,Chg Abs +1,375,985,4295,1400,850,412,0
2,Hilo < 2000,Chg Abs +1,748,981,4295,1400,825,392,0
3,Hilo < 2500,Chg Abs +1,984,984,4295,1400,828,395,0
4,Hilo < 3000,Chg Abs +1,1103,999,4440,1415,825,410,0
5,Hilo < 3500,Chg Abs +1,1185,1008,5150,1410,835,415,0
6,Hilo < 4000,Chg Abs +1,1213,1014,5150,1415,840,415,0
7,Hilo < 4500,Chg Abs +1,1229,1023,5150,1430,840,415,0
8,Hilo < 5000,Chg Abs +1,1238,1030,5150,1440,845,420,0
9,Hilo < 5500,Chg Abs +1,1241,1031,5150,1440,850,420,0


In [20]:
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Hilo < {1000}':f'Hilo < {7500}'].columns, _next=['Hilo +1'])

,Previous,Next,Count,Avg,Max,Q3,Med,Q1,Min
0,Hilo < 1000,Hilo +1,60,1742,3765,1986,1620,1328,690
1,Hilo < 1500,Hilo +1,375,1802,4545,2160,1670,1332,535
2,Hilo < 2000,Hilo +1,748,1841,4985,2195,1700,1354,535
3,Hilo < 2500,Hilo +1,984,1867,4985,2220,1730,1375,535
4,Hilo < 3000,Hilo +1,1103,1899,5950,2270,1740,1400,535
5,Hilo < 3500,Hilo +1,1185,1929,5950,2310,1770,1410,535
6,Hilo < 4000,Hilo +1,1213,1947,5950,2330,1775,1415,535
7,Hilo < 4500,Hilo +1,1229,1962,7400,2355,1785,1420,535
8,Hilo < 5000,Hilo +1,1238,1970,7400,2365,1790,1425,535
9,Hilo < 5500,Hilo +1,1241,1975,7400,2365,1790,1425,535


# Snippets

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])
    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if x >= 0 else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if x <  0 else nan)
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']

    BINS_CHG = [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, ...]
    BINS_HL  = [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, ...] 

    for x in BINS_CHG:  Calc[f'Chg Abs < {x}'] = (Calc['Chg Abs'] < x)
    for x in BINS_HL:   Calc[f'Hilo < {x}']    = (Calc['Hilo']    < x)

    FUTS = [1, 2]
    for x in FUTS:  Calc[f'Chg +{x}']       = Calc['Change'].shift(-x)
    for x in FUTS:  Calc[f'Chg Abs +{x}']   = Calc['Chg Abs'].shift(-x)
    for x in FUTS:  Calc[f'Chg Pos +{x}']   = Calc['Chg Pos'].shift(-x)
    for x in FUTS:  Calc[f'Chg Neg +{x}']   = Calc['Chg Neg'].shift(-x)

    for x in FUTS:  Calc[f'Hilo +{x}']      = Calc['Hilo'].shift(-x)
    for x in FUTS:  Calc[f'HL Pos +{x}']    = Calc['HL Pos'].shift(-x)
    for x in FUTS:  Calc[f'HL Neg +{x}']    = Calc['HL Neg'].shift(-x)
    return Calc

In [18]:
def MARKOV_CHAIN(Calc, when, _next):
    pipe = []
    for x in when: 
        Df = Calc[Calc[x]==1]

        for y in _next: 
            pipe.append({ 'Previous':x, 'Next':y, 
                'Count':          Df[y].count(), 
                'Avg':   np.round(Df[y].mean(),        0).astype(int),
                'Max':   np.round(Df[y].max(),         0).astype(int),
                'Q3':    np.round(Df[y].quantile(.75), 0).astype(int),
                'Med':   np.round(Df[y].median(),      0).astype(int),
                'Q1':    np.round(Df[y].quantile(.25), 0).astype(int),
                'Min':   np.round(Df[y].min(),         0).astype(int),
            })
        pass
    pass
    return pd.DataFrame(pipe)


MARKOV_CHAIN(Calc, when=Calc.loc[:,    f'Hilo < {1000}':   f'Hilo < {8000}'].columns, _next=['Hilo +1'])
MARKOV_CHAIN(Calc, when=Calc.loc[:,    f'Hilo < {1000}':   f'Hilo < {8000}'].columns, _next=['Chg Abs +1'])
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Chg Abs < {100}' :f'Chg Abs < {5500}'].columns, _next=['Hilo +1'])
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Chg Abs < {100}' :f'Chg Abs < {5500}'].columns, _next=['Chg Abs +1'])

,Previous,Next,Count,Avg,Max,Q3,Med,Q1,Min
0,Chg Abs < 100,Chg Abs +1,78,1016,3985,1509,878,322,60
1,Chg Abs < 200,Chg Abs +1,148,1010,3985,1431,890,419,5
2,Chg Abs < 500,Chg Abs +1,367,1097,4295,1568,935,458,5
3,Chg Abs < 800,Chg Abs +1,587,1042,4295,1475,885,438,0
4,Chg Abs < 1000,Chg Abs +1,720,1040,4295,1475,865,425,0
5,Chg Abs < 1200,Chg Abs +1,833,1036,4295,1450,865,425,0
6,Chg Abs < 1500,Chg Abs +1,953,1024,4295,1440,855,415,0
7,Chg Abs < 1800,Chg Abs +1,1039,1016,4295,1430,850,412,0
8,Chg Abs < 2000,Chg Abs +1,1095,1017,5150,1430,845,415,0
9,Chg Abs < 2200,Chg Abs +1,1125,1014,5150,1430,840,410,0
